### 1. Imports

In [39]:
from pathlib import Path

import numpy as np
import pandas as pd

### 2. Data Sourcing

In [40]:
# original / raw data source
lotto_csv_path = Path.joinpath(Path.cwd().parent, 'data\\raw\LOTTO6aus49_1990-2023.csv').resolve()
print(f'Lotto CSV file path: {lotto_csv_path}')

Lotto CSV file path: T:\repos\GamblingEvaluator\gamblingEvaluator\data\raw\LOTTO6aus49_1990-2023.csv


#### 2.1 Data Dictionary
| Column | Meaning 
| :--- | :--- 
| Year | Year of the drawing
| MM/DD | Month and day of the drawing
| Day | Drawing day, either SA (Saturday) or MI (Wednesday)
| GWZ1-6 | Winning numbers 1 to 6
| Zz | Additional winning number aka Zusatzzahl
| Sz | Additional winning number aka Superzahl. Sz fully replaced Zz in May 5, 2013
| Spieleinsatz | Total stake in EUR
| I | Quote; # of Winners in category I, the category with the largest possible profit  
| 6T_Zz | The single participant profit of category I, 6 correct winning numbers plus Zz/Sz
| II | Quote; # of Winners in category II
| 6T | The single participant profit of category II
| III | Quote; # of Winners in category III
| 5T_Zz | The single participant profit of category III
| IV | Quote; # of Winners in category IV
| 5T | The single participant profit of category IV
| V | Quote; # of Winners in category V
| 4T_Zz | The single participant profit of category V
| VI | Quote; # of Winners in category VI
| 4T | The single participant profit of category VI
| VII | Quote; # of Winners in category VII
| 3T_Zz | The single participant profit of category VII
| VIII | Quote; # of Winners in category VIII
| 3T | The single participant profit of category VIII
| IX | Quote; # of Winners in category IX, the category with the smallest possible profit
| 2T_ZZ | The single participant profit of category IX

<b>Note:</b><br>
Currency is EUR.<br>Until (and including) 29/12/2013 the currency used was DM.

##### 2.2 CSV Import

In [41]:
df = pd.read_csv(lotto_csv_path, sep=';', index_col=0)

### 3. Data Preprocessing

Column type conversions

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2978 entries, 1.0 to nan
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Year          2957 non-null   float64
 1   MM/DD         2957 non-null   object 
 2   Day           2957 non-null   object 
 3   GWZ1          2957 non-null   float64
 4   GWZ2          2957 non-null   float64
 5   GWZ3          2957 non-null   float64
 6   GWZ4          2957 non-null   float64
 7   GWZ5          2957 non-null   float64
 8   GWZ6          2957 non-null   float64
 9   Zz            2957 non-null   object 
 10  Sz            2957 non-null   object 
 11  Spieleinsatz  2957 non-null   object 
 12  I             2957 non-null   object 
 13  6T_Zz         2957 non-null   object 
 14  II            2957 non-null   object 
 15  6T            2957 non-null   object 
 16  III           2957 non-null   object 
 17  5T_Zz         2957 non-null   object 
 18  IV            2957 non-null   fl

#### 3.0 Remove empty entries

Conversions from Excel (our original data source) might lead to empty rows during conversion. Those need to be removed. 

In [43]:
ds_has_empty_rows = df.isna().any().iloc[0]
print(f'Empty rows in data set: {ds_has_empty_rows}')
if ds_has_empty_rows:
    # drow rows in place that are all na
    df.dropna(axis=0, how='all', inplace=True) 
print(f'Empty rows in data set (final): {ds_has_empty_rows}')

Empty rows in data set: True
Empty rows in data set (final): True


#### 3.1 Year column

In [44]:
df.Year = df.Year.astype(np.ushort)
df.Year.info()

<class 'pandas.core.series.Series'>
Index: 2957 entries, 1.0 to 42.0
Series name: Year
Non-Null Count  Dtype 
--------------  ----- 
2957 non-null   uint16
dtypes: uint16(1)
memory usage: 28.9 KB


#### 3.2 MM/DD column

In [45]:
# Note: Most likely not needed.

#### 3.3 Day column

In [46]:
df.Day = df.Day.astype('category')
df.Day.info()

<class 'pandas.core.series.Series'>
Index: 2957 entries, 1.0 to 42.0
Series name: Day
Non-Null Count  Dtype   
--------------  -----   
2957 non-null   category
dtypes: category(1)
memory usage: 26.1 KB


#### 3.4 GWZx, Zz, and Sz columns

In [47]:
gwz_zz_sz_columns = ['GWZ1', 'GWZ2', 'GWZ3', 'GWZ4', 'GWZ5', 'GWZ6', 'Zz', 'Sz']

Replace 'na' to np.NaN for quote columns

In [48]:
# get number of observations with no entries ('na' entries), e.g. because the winning category did not exist yet in the beginning
df[df[gwz_zz_sz_columns]=='na'][gwz_zz_sz_columns].count()

GWZ1       0
GWZ2       0
GWZ3       0
GWZ4       0
GWZ5       0
GWZ6       0
Zz      1092
Sz       100
dtype: int64

In [49]:
# set 'na' entries to numpy.NaN
df.loc[:, gwz_zz_sz_columns] = df.loc[:, gwz_zz_sz_columns].replace('na', np.NaN, inplace=False)
# verify that entries have been replaced (and no 'na' entries exist)
df[df[gwz_zz_sz_columns]=='na'][gwz_zz_sz_columns].count()

GWZ1    0
GWZ2    0
GWZ3    0
GWZ4    0
GWZ5    0
GWZ6    0
Zz      0
Sz      0
dtype: int64

In [50]:
# replace numpy.NaN entries of GWZx with zeros, due to semantic equivalence with na (0 is not allowed for GWZx)
df.loc[:, gwz_zz_sz_columns[:-2]] = df.loc[:, gwz_zz_sz_columns].fillna(0, axis=1, inplace=False)
# replace numpy.NaN entries of Sz and Sz with -1 (0 is allowed for Sz)
df.loc[:, gwz_zz_sz_columns[-2:]] = df.loc[:, gwz_zz_sz_columns].fillna(0, axis=1, inplace=False)

C:\Users\bruen\AppData\Local\Temp\ipykernel_9800\1999689314.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[44.0 37.0 8.0 ... 42.0 35.0 31.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, gwz_zz_sz_columns[:-2]] = df.loc[:, gwz_zz_sz_columns].fillna(0, axis=1, inplace=False)
C:\Users\bruen\AppData\Local\Temp\ipykernel_9800\1999689314.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[45.0 33.0 42.0 ... 30.0 31.0 16.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, gwz_zz_sz_columns[:-2]] = df.loc[:, gwz_zz_sz_columns].fillna(0, axis=1, inplace=False)
C:\Users\bruen\AppData\Local\Temp\ipykernel_9800\1999689314.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Valu

In [51]:
# convert to final (target) data type (integer)
df[gwz_zz_sz_columns] = df[gwz_zz_sz_columns].astype(np.uintc)
df[gwz_zz_sz_columns].info()

<class 'pandas.core.frame.DataFrame'>
Index: 2957 entries, 1.0 to 42.0
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   GWZ1    2957 non-null   uint32
 1   GWZ2    2957 non-null   uint32
 2   GWZ3    2957 non-null   uint32
 3   GWZ4    2957 non-null   uint32
 4   GWZ5    2957 non-null   uint32
 5   GWZ6    2957 non-null   uint32
 6   Zz      2957 non-null   uint32
 7   Sz      2957 non-null   uint32
dtypes: uint32(8)
memory usage: 115.5 KB


#### 3.5 Quote columns (quote ~ # winners)

In [52]:
quote_columns = ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX']

Some data types are incorrect. This can be explained e.g. by the empty (na) rows.<br>NA values for example cannot be converted to integer but float64.<br>One such example is the Year column.<br><br>1. Set na values of certain columns to 0 due to equivalent semantic meaning.<br>2. Convert int columns from float to int.

In [53]:
# remove all potential blanks for quote columns of type object (string)
quote_str_columns = ['I', 'II', 'III', 'V', 'VII', 'IX']
df.loc[:, quote_str_columns] = df.loc[:, quote_str_columns].apply(lambda x: x.str.strip())

Replace 'na' to np.NaN for quote columns

In [54]:
# get number of observations with no entries ('na' entries), e.g. because the winning category did not exist yet in the beginning
df[df[quote_columns]=='na'][quote_columns].count()

I        107
II        71
III        0
IV         0
V        489
VI         0
VII      100
VIII       0
IX      1865
dtype: int64

In [55]:
# set 'na' entries to numpy.NaN
df.loc[:, quote_columns] = df.loc[:, quote_columns].replace('na', np.NaN, inplace=False)
# verify that entries have been replaced (and no 'na' entries exist)
df[df[quote_columns]=='na'][quote_columns].count()

I       0
II      0
III     0
IV      0
V       0
VI      0
VII     0
VIII    0
IX      0
dtype: int64

In [56]:
# replace numpy.NaN entries with zeros, due to semantic equivalence
df.loc[:, quote_columns] = df.loc[:, quote_columns].fillna(0, axis=1, inplace=False)

C:\Users\bruen\AppData\Local\Temp\ipykernel_9800\2836301153.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[949.0 2217.0 1490.0 ... 300.0 384.0 235.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, quote_columns] = df.loc[:, quote_columns].fillna(0, axis=1, inplace=False)
C:\Users\bruen\AppData\Local\Temp\ipykernel_9800\2836301153.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[120914.0 102278.0 83043.0 ... 17291.0 26454.0 13698.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, quote_columns] = df.loc[:, quote_columns].fillna(0, axis=1, inplace=False)
C:\Users\bruen\AppData\Local\Temp\ipykernel_9800\2836301153.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. V

Replace 'JP' with '1.0' for quote columns, due to semantic equality

In [57]:
# get number of observations with JP (JackPot) entries
df[df[quote_columns]=='JP'][quote_columns].count()

I       2036
II       327
III        1
IV         0
V          0
VI         0
VII        0
VIII       0
IX         0
dtype: int64

In [58]:
# set 'na' entries to numpy.NaN
df.loc[:, quote_columns] = df.loc[:, quote_columns].replace('JP', 1.0, inplace=False)
# verify that entries have been replaced (and no 'na' entries exist)
df[df[quote_columns]=='JP'][quote_columns].count()

I       0
II      0
III     0
IV      0
V       0
VI      0
VII     0
VIII    0
IX      0
dtype: int64

Convert quote columns (str) to integer

In [59]:
# convert to final (target) data type (integer)
df[quote_columns] = df[quote_columns].astype(np.uintc)
df[quote_columns].info()

<class 'pandas.core.frame.DataFrame'>
Index: 2957 entries, 1.0 to 42.0
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   I       2957 non-null   uint32
 1   II      2957 non-null   uint32
 2   III     2957 non-null   uint32
 3   IV      2957 non-null   uint32
 4   V       2957 non-null   uint32
 5   VI      2957 non-null   uint32
 6   VII     2957 non-null   uint32
 7   VIII    2957 non-null   uint32
 8   IX      2957 non-null   uint32
dtypes: uint32(9)
memory usage: 127.1 KB


#### 3.6 Currency columns

In [60]:
currency_columns = ['Spieleinsatz', '6T_Zz', '6T', '5T_Zz', '5T', '4T_Zz', '4T', '3T_Zz', '3T', '2T_Zz']

Replace 'na' to np.NaN for quote columns

In [61]:
# get number of observations with no entries ('na' entries), e.g. because the winning category did not exist yet in the beginning
df[df[currency_columns]=='na'][currency_columns].count()

Spieleinsatz       0
6T_Zz            108
6T                71
5T_Zz              0
5T                 0
4T_Zz            489
4T                 0
3T_Zz            100
3T                 0
2T_Zz           1865
dtype: int64

In [62]:
# set 'na' entries to numpy.NaN
df.loc[:, currency_columns] = df.loc[:, currency_columns].replace('na', np.NaN, inplace=False)
# verify that entries have been replaced (and no 'na' entries exist)
df[df[currency_columns]=='na'][currency_columns].count()

Spieleinsatz    0
6T_Zz           0
6T              0
5T_Zz           0
5T              0
4T_Zz           0
4T              0
3T_Zz           0
3T              0
2T_Zz           0
dtype: int64

In [63]:
# replace numpy.NaN entries with zeros, due to semantic equivalence
df.loc[:, currency_columns] = df.loc[:, currency_columns].fillna(0, axis=1, inplace=False)

Convert quote columns (str) to integer

In [64]:
# convert to final (target) data type (integer)
df[currency_columns] = df[currency_columns].apply(lambda x: x.str.replace(',','.')).astype(np.float32)
df[currency_columns] = df[currency_columns].fillna(0, axis=1, inplace=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2957 entries, 1.0 to 42.0
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Year          2957 non-null   uint16  
 1   MM/DD         2957 non-null   object  
 2   Day           2957 non-null   category
 3   GWZ1          2957 non-null   uint32  
 4   GWZ2          2957 non-null   uint32  
 5   GWZ3          2957 non-null   uint32  
 6   GWZ4          2957 non-null   uint32  
 7   GWZ5          2957 non-null   uint32  
 8   GWZ6          2957 non-null   uint32  
 9   Zz            2957 non-null   uint32  
 10  Sz            2957 non-null   uint32  
 11  Spieleinsatz  2957 non-null   float32 
 12  I             2957 non-null   uint32  
 13  6T_Zz         2957 non-null   float32 
 14  II            2957 non-null   uint32  
 15  6T            2957 non-null   float32 
 16  III           2957 non-null   uint32  
 17  5T_Zz         2957 non-null   float32 
 18  IV         

In [65]:
df[currency_columns].head()

,Spieleinsatz,6T_Zz,6T,5T_Zz,5T,4T_Zz,4T,3T_Zz,3T,2T_Zz
Index,,,,,,,,,,
1.0,104514784.0,0.0,1.567722e+06,1.567722e+06,12389.700195,0.0,97.199997,0.0,10.5,0.0
2.0,105488112.0,0.0,6.593007e+05,6.593007e+05,5352.899902,0.0,116.000000,0.0,9.6,0.0
3.0,104143680.0,0.0,1.301796e+06,1.301796e+06,7863.100098,0.0,141.000000,0.0,10.2,0.0
4.0,103208216.0,0.0,8.600684e+05,8.600684e+05,5348.100098,0.0,98.699997,0.0,8.3,0.0
5.0,112148488.0,0.0,2.803712e+06,2.803712e+06,10914.099609,0.0,176.800003,0.0,12.5,0.0


In [66]:
df[currency_columns].tail()

,Spieleinsatz,6T_Zz,6T,5T_Zz,5T,4T_Zz,4T,3T_Zz,3T,2T_Zz
Index,,,,,,,,,,
40.0,22745274.0,1705895.5,1243710.50,22692.199219,6764.000000,265.000000,63.299999,25.100000,12.5,6.0
40.0,44378320.0,5034269.5,3739743.00,13956.299805,4532.899902,263.600006,52.700001,29.299999,11.5,6.0
41.0,23553136.0,11778089.0,0.00,11000.799805,4262.799805,169.500000,48.599998,18.299999,10.5,6.0
41.0,45335512.0,3400163.5,2352212.50,12943.299805,6329.700195,169.699997,60.400002,17.700001,12.1,6.0
42.0,22793734.0,5109693.5,3621788.75,17604.699219,5582.500000,263.899994,63.000000,28.000000,13.3,6.0


Currency adjustments (DM -> EUR)

In [67]:
# convert DM currency entries to Euro
dm_to_eur_conv_rate = 0.51129
df.loc[df.Year < 2002, currency_columns] = df.loc[df.Year < 2002, currency_columns].apply(lambda x: x * dm_to_eur_conv_rate)
df.head()

,Year,MM/DD,Day,GWZ1,GWZ2,GWZ3,GWZ4,GWZ5,GWZ6,Zz,...,V,4T_Zz,VI,4T,VII,3T_Zz,VIII,3T,IX,2T_Zz
Index,,,,,,,,,,,,,,,,,,,,,
1.0,1990,06/01/,SA,44,45,48,12,46,30,2,...,0,0.0,120914,49.697388,0,0.0,1607023,5.368545,0,0.0
2.0,1990,13/01/,SA,37,33,12,6,42,15,9,...,0,0.0,102278,59.309643,0,0.0,1772874,4.908384,0,0.0
3.0,1990,20/01/,SA,8,42,40,1,17,18,10,...,0,0.0,83043,72.091888,0,0.0,1648051,5.215158,0,0.0
4.0,1990,27/01/,SA,12,10,42,6,9,37,44,...,0,0.0,117530,50.464321,0,0.0,1999733,4.243707,0,0.0
5.0,1990,03/02/,SA,29,46,48,42,35,20,45,...,0,0.0,71328,90.396072,0,0.0,1446763,6.391125,0,0.0


Unify date format

In [68]:
df['MM/DD'] = df['MM/DD'].apply(lambda x: x.replace('.', '/'))

#### 3.7 Feature Engineering

In [69]:
df.columns

Index(['Year', 'MM/DD', 'Day', 'GWZ1', 'GWZ2', 'GWZ3', 'GWZ4', 'GWZ5', 'GWZ6',
       'Zz', 'Sz', 'Spieleinsatz', 'I', '6T_Zz', 'II', '6T', 'III', '5T_Zz',
       'IV', '5T', 'V', '4T_Zz', 'VI', '4T', 'VII', '3T_Zz', 'VIII', '3T',
       'IX', '2T_Zz'],
      dtype='object')

In [70]:
df['Total_payout'] = df['I'] * df['6T_Zz'] + df['II'] * df['6T'] + df['III'] * df['5T_Zz'] + df['IV'] * df['5T'] + df['V'] * df['4T_Zz'] + df['VI'] * df['4T'] + df['VII'] * df['3T_Zz'] + df['VIII'] * df['3T'] + df['IX'] * df['2T_Zz']

In [71]:
pd.to_datetime('16/11/1990', format='%d/%m/%Y')

Timestamp('1990-11-16 00:00:00')

In [72]:
df['Date'] = pd.to_datetime(df['MM/DD'] + df['Year'].apply(lambda x: str(x)), format='%d/%m/%Y')
df['Date']

Index
1.0    1990-01-06
2.0    1990-01-13
3.0    1990-01-20
4.0    1990-01-27
5.0    1990-02-03
          ...    
40.0   2023-10-04
40.0   2023-10-07
41.0   2023-10-11
41.0   2023-10-14
42.0   2023-10-18
Name: Date, Length: 2957, dtype: datetime64[ns]

In [73]:
df['gwzx_sum'] = df[gwz_zz_sz_columns[:6]].sum(axis=1)

#### 3.8 Reset index

In [74]:
df.index = df.index.astype(np.int16)
df.reset_index(drop=True, inplace=True)
df.tail(5)

,Year,MM/DD,Day,GWZ1,GWZ2,GWZ3,GWZ4,GWZ5,GWZ6,Zz,...,4T,VII,3T_Zz,VIII,3T,IX,2T_Zz,Total_payout,Date,gwzx_sum
2952,2023,04/10/,MI,28,38,18,19,46,43,0,...,63.299999,28652,25.100000,270592,12.5,229223,6.0,1.134449e+07,2023-10-04,192
2953,2023,07/10/,SA,23,32,11,22,42,14,0,...,52.700001,49283,29.299999,593407,11.5,370095,6.0,2.511946e+07,2023-10-07,144
2954,2023,11/10/,MI,42,30,3,25,16,31,0,...,48.599998,39033,18.299999,321396,10.5,293246,6.0,2.052950e+07,2023-10-11,147
2955,2023,14/10/,SA,35,31,10,22,8,29,0,...,60.400002,77007,17.700001,530896,12.1,597696,6.0,2.264349e+07,2023-10-14,135
2956,2023,18/10/,MI,31,16,15,47,42,45,0,...,63.000000,26290,28.000000,260428,13.3,203916,6.0,1.713369e+07,2023-10-18,196


### 4. Store preprocessed CSV

In [75]:
final_lotto_csv_path = Path.joinpath(Path.cwd().parent, 'data\\preprocessed\LOTTO6aus49_1990-2023.csv').resolve()

df.to_csv(final_lotto_csv_path)